In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
torch.cuda.is_available()

True

In [4]:
%config IPCompleter.greedy=True

In [5]:
train_set= torchvision.datasets.FashionMNIST(
    root= './data/FashionMNIST',
    train= True,
    download= True,
    transform= transforms.Compose([
        transforms.ToTensor()
    ])
)

In [6]:
import torch.nn as nn

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1= nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2= nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1= nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2= nn.Linear(in_features=120, out_features=60)
        self.out= nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # input layer
        t=t
        # hidden 1st convolutional layer
        t=self.conv1(t)
        t=nn.functional.relu(t)
        t=nn.functional.max_pool2d(t, k/ernel_size=2, stride=2)
        
        # hidden 2nd convolutional layer
        t=self.conv2(t)
        t=nn.functional.relu(t)
        t=nn.functional.max_pool2d(t, kernel_size=2, stride=2)        
        
        # flattening for linear layer
        t=t.reshape(-1, 12*4*4)
        
        # linear layer
        t=self.fc1(t)
        t=nn.functional.relu(t)

        t=self.fc2(t)
        t=nn.functional.relu(t)

        # output layer
        t=self.out(t)
        #t=F.softmax(t, dim=1)
        
        return t

    # def __repr__(self):
    #     return "Convolutional Neural Network"

In [7]:
network= Network()
network

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [8]:
data_loader= torch.utils.data.DataLoader(
    train_set,
    batch_size=10
)

In [9]:
batch= next(iter(data_loader))
images, labels= batch
print(images.shape, '\n', labels.shape)

torch.Size([10, 1, 28, 28]) 
 torch.Size([10])


In [10]:
preds= network(images)
print(preds)
print('labels= ', torch.argmax(preds, dim=1))

tensor([[ 0.0296,  0.0420, -0.1422, -0.0768, -0.0829,  0.0502,  0.1075, -0.1292,
         -0.0662,  0.0071],
        [ 0.0327,  0.0314, -0.1511, -0.0702, -0.0703,  0.0627,  0.1116, -0.1262,
         -0.0641, -0.0045],
        [ 0.0073,  0.0436, -0.1291, -0.0961, -0.0809,  0.0655,  0.1244, -0.1453,
         -0.0592, -0.0039],
        [ 0.0141,  0.0397, -0.1386, -0.0893, -0.0783,  0.0628,  0.1185, -0.1387,
         -0.0608, -0.0018],
        [ 0.0246,  0.0407, -0.1355, -0.0664, -0.0695,  0.0575,  0.1256, -0.1312,
         -0.0653, -0.0079],
        [ 0.0293,  0.0394, -0.1461, -0.0802, -0.0821,  0.0527,  0.1107, -0.1292,
         -0.0685,  0.0013],
        [ 0.0162,  0.0440, -0.1503, -0.0969, -0.0889,  0.0407,  0.0994, -0.1284,
         -0.0718,  0.0101],
        [ 0.0369,  0.0341, -0.1468, -0.0730, -0.0763,  0.0493,  0.1061, -0.1252,
         -0.0666,  0.0043],
        [ 0.0016,  0.0449, -0.1373, -0.1037, -0.0827,  0.0691,  0.1131, -0.1468,
         -0.0589, -0.0010],
        [ 0.0193,  

In [11]:
# comparision with the actual
torch.argmax(preds, dim=1).eq(labels)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)

In [12]:
# accuracy percentage
def accuracy(preds, labels):
    return ((torch.argmax(preds, dim=1).eq(labels).sum()).double()/ len(labels))
accuracy(preds, labels)

tensor(0., dtype=torch.float64)

In [13]:
# accuracy percentage
def accuracy(preds, labels):
    return ((torch.argmax(preds, dim=1).eq(labels).sum()).double()/ len(labels))
accuracy(preds, labels)

tensor(0., dtype=torch.float64)

In [15]:
# loss function
loss= nn.functional.cross_entropy(preds, labels)
loss.item()

2.3094775676727295

In [17]:
print(network.conv1.weight.grad)
loss.backward()
print(network.conv1.weight.grad)


None
tensor([[[[-0.0022, -0.0020, -0.0016, -0.0017, -0.0023],
          [-0.0026, -0.0027, -0.0038, -0.0039, -0.0037],
          [-0.0028, -0.0026, -0.0034, -0.0023, -0.0035],
          [-0.0024, -0.0029, -0.0034, -0.0035, -0.0019],
          [-0.0029, -0.0030, -0.0041, -0.0030, -0.0027]]],


        [[[ 0.0019,  0.0008,  0.0011,  0.0014,  0.0005],
          [ 0.0022,  0.0016,  0.0011, -0.0000, -0.0016],
          [ 0.0017,  0.0007,  0.0000, -0.0009, -0.0013],
          [ 0.0010,  0.0013,  0.0015,  0.0007,  0.0002],
          [ 0.0015,  0.0009,  0.0010,  0.0017,  0.0008]]],


        [[[-0.0014, -0.0009, -0.0016, -0.0024, -0.0025],
          [-0.0012, -0.0012, -0.0020, -0.0025, -0.0031],
          [-0.0006, -0.0004, -0.0007, -0.0012, -0.0020],
          [-0.0003,  0.0004, -0.0002, -0.0010, -0.0020],
          [-0.0003, -0.0004, -0.0018, -0.0026, -0.0035]]],


        [[[ 0.0020,  0.0016, -0.0010, -0.0030, -0.0042],
          [ 0.0010,  0.0001, -0.0018, -0.0023, -0.0028],
          [ 0.

In [32]:
# Summary
from torch import optim

# train dataset download
train_set= torchvision.datasets.FashionMNIST(
    root= './data/FashionMNIST',
    train= True,
    download= True,
    transform= transforms.Compose([
        transforms.ToTensor()
    ])
)

# network instance
network= Network()

# train dataset loader
train_loader= torch.utils.data.DataLoader(train_set, batch_size= 10)

# optimizer function
optimizer= optim.Adam(network.parameters(), lr=0.01)

# load the first batch and iterate there after
batch = next(iter(data_loader))
images, labels= batch

# prediction before anything
preds= network(images)
print('previous prediction', accuracy(preds, labels))

# initialize the loss function
loss= nn.functional.cross_entropy(preds, labels)
print('previous loss:', loss.item())

# initialize backward propagation
loss.backward() # calculate gradients
optimizer.step() # update weights

# prediction after updating weights
preds= network(images)
loss= nn.functional.cross_entropy(preds, labels)
print('new loss:', loss.item())
print('new prediction:', accuracy(preds, labels))


previous prediction tensor(0.1000, dtype=torch.float64)
previous loss: 2.315969467163086
new loss: 2.240274429321289
new prediction: tensor(0.2000, dtype=torch.float64)
